## Import the libraries

In [1]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from pathlib import Path

## load and preprocess the Dataset

In [2]:
# Load data set
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [3]:
# Normalize data set to 0-to-1 range
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [4]:
# Convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)


## Building the model

In [5]:
# Create a model and add layers
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', input_shape=(32, 32, 3), activation="relu"))
model.add(Conv2D(32, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation="relu"))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation="softmax"))


In [6]:
# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

## Training_and_saving

In [ ]:
# Train the model
model.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=30,
    validation_data=(x_test, y_test),
    shuffle=True
)


In [ ]:
# Save neural network structure
model_structure = model.to_json()
f = Path("model_structure.json")
f.write_text(model_structure)

In [ ]:
# Save neural network's trained weights
model.save_weights("model_weights.h5")

#                                      Making predictions

## Import dependencies:

In [ ]:
from keras.models import model_from_json
from pathlib import Path
from keras.preprocessing import image
import numpy as np

In [ ]:
# These are the CIFAR10 class labels from the training data (in order from 0 to 9)
class_labels = [
    "Plane",
    "Car",
    "Bird",
    "Cat",
    "Deer",
    "Dog",
    "Frog",
    "Horse",
    "Boat",
    "Truck"
]


## Recalling the saved structure of the network

In [ ]:
# Load the json file that contains the model's structure
f = Path("model_structure.json")
model_structure = f.read_text()

In [ ]:
# Recreate the Keras model object from the json data
model = model_from_json(model_structure)

## Recalling the saved weights

In [ ]:
# Re-load the model's trained weights
model.load_weights("model_weights.h5")

## Make a prediction for one image

In [ ]:
# Load an image file to test, resizing it to 32x32 pixels (as required by this model)
img = image.load_img("frog.png", target_size=(32, 32))

In [ ]:
# Convert the image to a numpy array
image_to_test = image.img_to_array(img)

In [ ]:
# Add a fourth dimension to the image (since Keras expects a list of images, not a single image)
list_of_images = np.expand_dims(image_to_test, axis=0)


In [ ]:
# Make a prediction using the model
results = model.predict(list_of_images)


In [ ]:
# Since we are only testing one image, we only need to check the first result
single_result = results[0]

In [ ]:
# We will get a likelihood score for all 10 possible classes. Find out which class had the highest score.
most_likely_class_index = int(np.argmax(single_result))
class_likelihood = single_result[most_likely_class_index]

In [ ]:
# Get the name of the most likely class
class_label = class_labels[most_likely_class_index]

## Show the result

In [ ]:
# Print the result
print("This is image is a {} - Likelihood: {:2f}".format(class_label, class_likelihood))